In [1]:
# Load all packages
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [47]:
# Create dataframe
df1 = pd.read_csv("../../Data/20190403/Datalab_Reis_Bestemming_Datum_20190402.csv", sep=";")
df1 = df1.dropna()
df2 = pd.read_csv("../../Data/20190403/Datalab_Reis_Bestemming_Uur_20190402.csv", sep=";")
df2 = df2.dropna()
df3 = pd.read_csv("../../Data/20190403/Datalab_Reis_Herkomst_Datum_20190402.csv", sep=";")
df3 = df3.dropna()
df4 = pd.read_csv("../../Data/20190403/Datalab_Reis_Herkomst_Uur_20190403.csv", sep=";")
df4 = df4.dropna()
df5 = pd.read_csv("../../Data/20190403/Datalab_Rit_Bestemming_Uur_20190403.csv", sep=";")
df5 = df5.dropna()
df6 = pd.read_csv("../../Data/20190403/Datalab_Rit_Herkomst_Bestemming_Uur_20190403.csv", sep=";")
df6 = df6.dropna()
df7 = pd.read_csv("../../Data/20190403/Datalab_Rit_Herkomst_Uur_20190403.csv", sep=";")
df7 = df7.dropna()

In [59]:
df4[(df4["VertrekHalteNaam"] == "Noord") 
    & (df4["UurgroepOmschrijving (van vertrek)"] == "19:00 - 19:59") 
    & (df4["Datum"] == "3/24/2019 12:00:00 AM")]

,Datum,UurgroepOmschrijving (van vertrek),VertrekHalteCode,VertrekHalteNaam,VertrekLat,VertrekLon,AantalReizen
3491792,3/24/2019 12:00:00 AM,19:00 - 19:59,ND,Noord,4.931232,52.401172,462


In [60]:
df2[(df2["AankomstHalteNaam"] == "Noord") 
    & (df2["UurgroepOmschrijving (van aankomst)"] == "19:00 - 19:59") 
    & (df2["Datum"] == "3/24/2019 12:00:00 AM")]

,Datum,UurgroepOmschrijving (van aankomst),AankomstHalteCode,AankomstHalteNaam,AankomstLat,AankomstLon,AantalReizen
3503843,3/24/2019 12:00:00 AM,19:00 - 19:59,ND,Noord,4.931232,52.401172,835


In [62]:
df6[(df6["AankomstHalteNaam"] == "Noord") 
    & (df6["UurgroepOmschrijving (van vertrek)"] == "19:00 - 19:59") 
    & (df6["Datum"] == "3/24/2019 12:00:00 AM")]

,Datum,UurgroepOmschrijving (van vertrek),VertrekHalteCode,VertrekHalteNaam,VertrekLat,VertrekLon,AankomstHalteCode,AankomstHalteNaam,AankomstLat,AankomstLon,AantalRitten
9608177,3/24/2019 12:00:00 AM,19:00 - 19:59,CS,Centraal Station,4.900781,52.377494,ND,Noord,4.931232,52.401172,560
9608203,3/24/2019 12:00:00 AM,19:00 - 19:59,DPP,De Pijp,4.891011,52.352962,ND,Noord,4.931232,52.401172,41
9608254,3/24/2019 12:00:00 AM,19:00 - 19:59,ND,Noord,4.931232,52.401172,ND,Noord,4.931232,52.401172,25
9608264,3/24/2019 12:00:00 AM,19:00 - 19:59,NDP,Noorderpark,4.917891,52.388555,ND,Noord,4.931232,52.401172,22
9608301,3/24/2019 12:00:00 AM,19:00 - 19:59,RKN,Rokin,4.892297,52.369280,ND,Noord,4.931232,52.401172,159
9608334,3/24/2019 12:00:00 AM,19:00 - 19:59,VZG,Vijzelgracht,4.890729,52.359801,ND,Noord,4.931232,52.401172,68
9608383,3/24/2019 12:00:00 AM,19:00 - 19:59,WTC,Station Zuid,4.875099,52.339080,ND,Noord,4.931232,52.401172,37


In [67]:
iets = df6[df6["AankomstHalteCode"] == "RKN"]